In [ ]:
import numpy as np
import scipy
import pandas
import matplotlib.pyplot as plt

# Bajamos los datos

In [ ]:
# !wget -O datos.zip https://www.dropbox.com/scl/fi/v6qfj1ktarocr8sl02r8k/datos.zip?rlkey=2u060s5619gvcvnnnhq93rn4e&st=jy3dah88&dl=1
# !unzip datos.zip

In [ ]:
class_names = ["T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

# Cargamos los archivos

In [ ]:
X_train = np.loadtxt("datos/X_train.csv", delimiter=",") #Funcón de numpy que carga texto (En nuestro caso las imagenes)
y_train = np.loadtxt("datos/y_train.csv", delimiter=",").astype(int) # Indice entero de que clase es cada imagen, hay 5000
X_test = np.loadtxt("datos/X_test.csv", delimiter=",") # Variables test son las de prueba, NO USAR
y_test = np.loadtxt("datos/y_test.csv", delimiter=",").astype(int) # Indice de las imagenes de PRUEBA

X_train.shape, y_train.shape, X_test.shape, y_test.shape

# Revisamos la cantida de muestras por cada clase

In [ ]:
np.bincount(y_train), np.bincount(y_test) # Cuenta cuántos ceros hay, cuántos unos hay, etc.. Hay 500 de cada uno en los datos de entrenamiento y 50 de cada uno en los datos de prueba

# Visualizamos

In [ ]:
ix = 7 #Agarro el elemento numero 7
plt.imshow(X_test[ix].reshape(28,28), cmap="gray") #Reshape para acomodarlo en 28 * 28 (cuadrado)
plt.title(class_names[y_test[ix]]); #Le pone el nombre ... chequea que está bien (coincide el titulo con la imagen)

# Separamos en datos de desarrollo

Garantizamos que las clases siguen igualmente balanceadas (estratificación)

In [ ]:
from sklearn.model_selection import train_test_split
X_newtrain, X_dev, y_newtrain, y_dev = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train) # Test size = 20%, divide un quinto para reservar como desarrollo.
#Stratify garantiza que todo lo que metió en ese 20% tiene la misma cantidad de prendas de ropa que en el resto

np.bincount(y_newtrain), np.bincount(y_dev) #New Train = E , y_dev = Desarrollo

# 1.KNN

### Implementación en Python

In [ ]:
def distance(A, B):
    dot_product = A.dot(B)
    norm_product = np.linalg.norm(A) * np.linalg.norm(B)
    return 1 - (dot_product / norm_product)


#image set = lista de tuplas, con cada tupla = (imagen(vector), typo de imagen)
def image_distances(image_set, image):
    distances = []
    dis = 0
    for pic, label in image_set:
        dis = distance(image, pic)
        distances.append((dis, label))
    return distances

#distances = lista de tuplas, con cada tupla = (distanca de cada imagen de entrenamiento con la imagen elegida, typo de imagen)
def classify_image(distances, k):
    distances.sort(key=lambda x: x[0])
    number_of_apperances = np.zeros(10)
    for i in range(0, k): # k = cantidad de vecinos que queremos analizar 
       image_type = distances[i][1]
       number_of_apperances[image_type] = number_of_apperances[image_type] + 1  

    max = -1
    max_index = -1
    for i in range(0, len(number_of_apperances)):
        if(number_of_apperances[i] > max):
            max = number_of_apperances[i]
            max_index = i
    
    return max_index

def first_test(image, image_type):
    images = X_train[:4000]
    types = y_train[:4000]
    image_set = zip(images, types) #creamos las tuplas
    distances = image_distances(image_set, image)
    expected_type = classify_image(distances, 5)
    return expected_type
    # print(f"succes: {image_type == expected_type}; original type: {image_type}; expected type: {expected_type}")

expected_types = []
for i in range(4000, 5000):
    image = X_train[i]
    image_type = y_train[i]
    expected_types.append((first_test(image, image_type), image_type))

print(expected_types)

In [ ]:
def distanceMatrix(A, B):   # Apilar verticalmente los vectores en una matriz
    # Calcular las normas de cada vector de las matrices A y B
    norms_a = np.linalg.norm(A, axis=1)
    norms_b = np.linalg.norm(B, axis=1)
    # Dividir cada componente de las matrices por la norma del vector correspondiente
    A_normalized = A / norms_a[:, np.newaxis]
    B_normalized = B / norms_b[:, np.newaxis]
    # Calcula la matriz traspuesta de B
    B_normalized_t = np.transpose(B_normalized)
    # Calcula la distancias haciendo una multiplicació n entre ambas matrices
    distances = A_normalized @ B_normalized_t
    # Hace 1 - distances
    # ones = np.ones((len(distances), len(distances)))
    distances = 1 - distances
    
    return distances

#######REVISAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAR ESTE IF NO ES LA MEJOR IMPLEMENTACION
def knn(k, distances_matrix, start_index_desarrollo, end_index_desarrollo):
    # Recorta la matriz segun nuestra conveniencia 
    sub_matrix = distances_matrix[start_index_desarrollo:end_index_desarrollo+1]
    # Eliminar las columnas desde col_start_index hasta col_end_index (inclusive col_end_index)
    #######REVISAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAR ESTE IF NO ES LA MEJOR IMPLEMENTACION
    if not(start_index_desarrollo == 0 and end_index_desarrollo == len(distances_matrix)):
        sub_matrix = np.delete(sub_matrix, np.s_[start_index_desarrollo:end_index_desarrollo+1], axis=1)
    # Ordena por distancias de más cercana a más lejana y hace módulo 10 
    sub_matrix = np.argsort(sub_matrix, axis=1) % 10
    # Se queda con las primeras k columnas más cercanas
    k_sub_matrix = sub_matrix[:, :k]
    # Calcula la moda 
    mode, frecuency = scipy.stats.mode(k_sub_matrix, axis=1)
    return mode

print(knn(5,distanceMatrix(X_train, X_train),4000,5000))


In [ ]:
#test matriz distancias
def second_test():
    distancias_clasica = np.zeros((5000, 5000))
    for i in range(0, 5000):
        for j in range(0, 5000): 
            distancias_clasica[i][j] = distance(X_train[i], X_train[j])
    return distancias_clasica


def son_iguales(distancias_clasica):
    distance_maitrx = distanceMatrix(X_train, X_train )
    matriz_de_comparacion = np.full((5000, 5000), False, dtype=bool)
    for i in range(0, 5000):
        for j in range(0, 5000):
            if np.isclose(distancias_clasica[i][j], distance_maitrx[i][j], atol=0.00001):
                matriz_de_comparacion[i][j] = True
    # print(distance_matrix)
    print(matriz_de_comparacion)

# second_test()
old_distances = second_test()
son_iguales(old_distances)

## 3. Reconocimiento de imágenes en Fashion MNist
    

#### a - Realizar un reconocedor de imágenes usando KNN para un k fijo de 5, usando los datos de entrenamiento dados y medir la performance con la medida de exactitud en el conjunto de prueba. 

In [ ]:
def image_recognizer():
    # Calculamos las distancias entre las imágenes en X_ train y  X_test
    distances_matrix = distanceMatrix(X_test, X_train)
    # Hacemos KNN, siendo los elementos de prueba los que van de 5001 al 5500
    res_knn = knn(5,distances_matrix, 0, len(distances_matrix)) 

    # Para medir la performance, comparo el resultado de knn (modas) contra el resultado real (y_test)
    correctos = np.sum(res_knn == y_test) # Calcula la cantidad de elementos que son iguales en la misma posición
    performance = correctos / 500

    print("Predicciones correctas =", correctos)
    print("Performance =", performance)

image_recognizer()

#### b - Explorar el hiperparámetro k, usando 5-fold cross-validation con el conjunto de entrenamiento

In [ ]:
# Implementación en Python
# def calculate_With_traingin_development(training, development, k):
#     ammount_of_succeses = 0
#     for image in development:
#         distances = image_distances(training, image)
#         expected_type = classify_image(distances, k)
#         if (expected_type == image[1]):
#             ammount_of_succeses = ammount_of_succeses + 1
#     return ammount_of_succeses / 1000

# for i in range(0, 5):
#     full_image_set = list(zip(X_train, y_train))
#     inicio_D = i * 1000
#     fin_D = inicio_D + 1000
    
#     # Creando la lista D
#     D = full_image_set[inicio_D:fin_D]
    
#     # Creando la lista T
#     T = full_image_set[:inicio_D] + full_image_set[fin_D:]

#     calculate_With_traingin_development(T, D, 5)
   

In [ ]:
# Revisar creo que está mal xq los resultados dan raro
def cross_validation():
    precomputo = distanceMatrix(X_train, X_train)
    averages = []
    for k in range (1,10): 
        performances = []
        for i in range (0,5):
            start = i*1000
            end = (i+1)*1000
            res_knn = knn(k, precomputo, start, end)
            # Calcula la cantidad de elementos calculados que coinciden con su tipo original
            correctos = np.sum(res_knn == y_train[start:end+1])
            performance = correctos / 1000
            performances.append(performance)
        average = np.mean(performances)
        averages.append((k, average))

    best_k_index = np.argmax([avg[1] for avg in averages])
    best_k = averages[best_k_index][0]
    return best_k

cross_validation()


#### c - Preprocesar los datos de entrenamiento con PCA. Visualizar la cantidad de varianza explicada en función de la cantidad de componentes p


In [ ]:
# Se resta a cada valor la media de la imagen a la que pertenece (por fila)
X_centered = X_test - np.mean(X_test, axis=1)

# Se calcula la matriz de covarianza
covariance_matrix = np.cov(X_centered)

# Encontrar autovalores y autovectores USANDO EL MÉTODO DE LA POTENCIA 
eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

# V matriz de autovectores , D matriz de autovalores (ordenarlas por importancia)

# Reducir la dimensionalidad de los datos a p componentes = mult X_train por V utilizando los primeros p vectores columna de la matriz V

# Calcular la varianza explicada en función de las p componentes   

# Hacerlo para distintos valores de P

#### d - Pipeline final: Exploración conjunta de los hiperparámetros k de KNN, y p de PCA.


pybind11 está instalado


# Método de la potencia

In [ ]:
# !rm metodo_potencia.so 
# !g++ -shared =fPIC -Llibdl -o metodo_potencia.so metodo_potencia.cpp
# !ls 

In [ ]:
# import ctypes

# class sharedlib():
#     dlclose = ctypes.CDLL(None).dlclose  # This WON'T work on Win
#     dlclose.argtypes = (ctypes.c_void_p,)

#     def __init__(self, path, method, *args):
#         self.lib = ctypes.cdll.LoadLibrary(f'./{path}')

#         # Se explicitan los tipos de los argumentos para el método deseado
#         self.method_object = getattr(self.lib, method)
#         self.method_object.argtypes = args

#     def __call__(self, *args):
#         return self.method_object(*args)

#     def unload(self):
#         while self.dlclose(self.lib._handle)!=-1:
#             pass

# lib = sharedlib('metodo_potencia.so', 'power_iteration_deflation',
#                                                                 ctypes.POINTER(ctypes.c_double),
#                                                                 ctypes.POINTER(ctypes.c_double),
#                                                                 ctypes.POINTER(ctypes.c_double),
#                                                                 ctypes.c_int,
#                                                                 ctypes.c_int
#                                                             )

# # Creamos la matriz de entrada
# A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float64)
# # Eigen la mapea transpuesta a la información del puntero así que hay que pasarlo a orden tipo Fortran
# A = np.asfortranarray(A)
# b = np.array([1, 2, 3], dtype=np.float64)

# # Definimos el vector donde se van a guardar los resultados
# result = np.zeros((3,), dtype=np.float64)

# # Llamamos a nuestra función en C++ pasando los argumentos de entrada
# lib(
#     A.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
#     b.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
#     result.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
#     ctypes.c_int(A.shape[0]),
#     ctypes.c_int(A.shape[1])
# )

# lib.unload() # para poder recompilar la lib hay que cerrarla

# result